### RULES
Requires format of skipped line between different phylums (ex. empty row above Diatom, Dinoflagellate, etc.)

Assumed all {Ochromonas, } are mixotrophs.

1. assume everything after "Unknown flagellates" is irrelevant (to be deleted)
2. diatoms are NOT mixotrophs
3. remove all "[name]-like" (without genus specified)
4. remove all "[genus name] spp." AND "[genus name] sp."
5. check "cysts of"

Status Key--  
Confirmed := explicitly in the Mixotroph Database  
Unsure (sp. in mdb) := genus in Mixotroph Database lists "[genus name] sp." (ex. Ochromonas sp. for Ochromonas danica)  
Unsure (inexact name):= LIS name is in a longer Mixotroph Database name or vice versa (ex. Chattonella marina in Chattonella marina var. ovata)   

### QUESTIONS TO ASK

1. Should I be considering "cysts of Linggulodinium polyedrum" mixotrophs?
2. How should I handle these "unsure" cases? - see status key above

In [40]:
import pandas as pd
import numpy as np

from datetime import datetime

pd.set_option("future.no_silent_downcasting", True)

In [66]:
mdb = pd.read_csv("MDB - 3Dec2022.csv")
mdb.columns = mdb.iloc[1]
mdb = mdb.drop([0, 1]).reset_index(drop=True)

# edit mdb so that species ending in "sp" now end in "sp."
mdb['Species Name'] = mdb['Species Name'].str.replace(r'sp$', 'sp.', regex=True)

mdb.head()

1,Species Name,Taxonomic Group,AphiaID,Additional notes,Gene markers,PR2 Accession Number,GenBank Accession Number,Reference to sequence,MFT,Evidence of mixoplankton activity,...,REDS,SANT,SARC,SATL,SPSG,SSTC,SUND,TASM,WARM,WTRA
0,Acanthochiasma sp.,Radiolaria,368427,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,HM103395.1.1099_U;HM103418.1.1104_U;JN811207.1...,HM103395;HM103418;JN811207;GU825020;HM103399;H...,"Quaiser,A.. Comparative metagenomics of bathyp...",eSNCM,endosymbionts,...,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded
1,Acanthometra fusca,Radiolaria,not registered,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,KC172856.1.1696_U;EU446351.1.1552_U;JN811165.1...,KC172856;EU446351;JN811165,"Decelle,J.. Diversity, ecology and biogeochemi...",eSNCM,endosymbionts,...,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded
2,Acanthodesmia vinculata,Radiolaria,493675,Acantharia,not recorded,not recorded,not recorded,not recorded,eSNCM,endosymbionts,...,not recorded,not recorded,not recorded,2,8,not recorded,not recorded,not recorded,15,not recorded
3,Acanthometra pellucida,Radiolaria,235750,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,JN811196.1.1668_U;JQ697712.1.1693_U;JQ697708.1...,JN811196;JQ697712;JQ697708;JN811190;JQ697711;J...,"Decelle,J.. Molecular Phylogeny and Morphologi...",eSNCM,endosymbionts,...,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded
4,Acanthometron sp.,Radiolaria,391880,Acantharia,not recorded,not recorded,not recorded,not recorded,eSNCM,endosymbionts,...,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded


In [42]:
csv_name = "LIS_2019-Phytoplankton_Final Report Data.xlsx - 2019 LIS phytoplankton count"

In [43]:
# import and clean LIS data
lis = pd.read_csv(f"inputs/{csv_name}.csv")
original_headers = lis.columns  # save original column headers

phylum_ind = lis[lis.iloc[:, 0] == "Phylum"].index[0]
lis.columns = lis.iloc[phylum_ind]  # reset column headers
lis = lis.iloc[phylum_ind+2:].reset_index(drop=True)  

In [44]:
# remove rows after unknown flagellates
unknown_flagellates_ind = lis[lis["Phylum"] == "Unknown flagellates"].index[0] 
lis = lis.iloc[:unknown_flagellates_ind]
lis = lis.iloc[:lis.last_valid_index()+1]  # remove trailing nan rows

In [45]:
# remove rows that contain "TOTAL"
lis = lis[~lis["Phylum"].str.contains("TOTAL", na=False)].reset_index(drop=True)  

In [46]:
# construct correct phylum column
actual_phylum_ind = lis[lis["Species"].isna() & lis["Phylum"].isna()].index + 1
lis = lis.rename(columns={"Phylum": "Genus"}) # rename phylum column to genus
lis.insert(0, 'Phylum', lis["Genus"].iloc[actual_phylum_ind])  # reconstruct phylum column
lis['Phylum'] = lis['Phylum'].ffill()  # forwardfill phylum

lis['Genus'] = lis['Species'].str.split().str[0]  # fill genus using first word of species name

lis = lis.dropna(subset=['Species']).reset_index(drop=True) # delete rows with na in Species column

SPECIES_COL = 3
lis.iloc[:, SPECIES_COL+1:] = lis.iloc[:, SPECIES_COL+1:].replace(",", "", regex=True).astype(float)  # ensure numerical values are floats

In [47]:
# add Status column
lis.insert(0, 'Status', None)

In [48]:
# store blocks of known mixotroph genuses 
ochromonas_ind = lis[lis["Species"].str.contains("Ochromonas")].index
ochromonas_block = lis.iloc[ochromonas_ind] 

In [49]:
# remove based on hard coded rules (NOT RESETTING INDEX IN ORDER TO ADD BLOCKS BACK CORRECTLY)
lis = lis[lis["Phylum"] != "Diatom"] # remove all diatoms
lis = lis[~lis["Species"].str.contains("-like")] # remove species ending with "-like"
lis = lis[~lis["Species"].str.contains("sp.|spp.")]  # remove all sp. / spp.

In [50]:
# check "cysts of"
CYSTS_LEN = len("cysts of ")
cysts_of = lis[lis["Species"].str.contains("cysts of", regex=False)]["Species"].str.slice(CYSTS_LEN)
filtered = cysts_of.isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Confirmed"

In [51]:
# add back stored blocks of known mixotrophs and mark as Confirmed
lis = pd.concat([lis, ochromonas_block]).sort_index().drop_duplicates()
lis.loc[ochromonas_ind, "Status"] = "Confirmed"

In [52]:
# check if (in none status) direct match and mark all Trues as "Confirmed"
filtered = lis[lis['Status'].isnull()]["Species"].isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Confirmed"

# check (in remaining none status) if the genus has sp. and mark all Trues as "Unsure (sp. in mdb)"
genus_to_check = lis[lis['Status'].isnull()]['Species'].str.split().str[0].drop_duplicates() + " sp."
filtered = genus_to_check.isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Unsure (sp. mdb)"

In [53]:
# check (in remaining none status) if the name is contained in the mdb and vice versa and mark all Trues as "Unsure (inexact name)"
filtered = lis[lis['Status'].isnull()]["Species"].apply(lambda x: mdb["Species Name"].str.contains(x, regex=False).any())
lis.loc[filtered[filtered].index, "Status"] = "Unsure (inexact name)"

pattern = '|'.join(mdb['Species Name'])
filtered = lis[lis['Status'].isnull()]["Species"].str.contains(pattern, regex=True)
lis.loc[filtered[filtered].index, "Status"] = "Unsure (inexact name)"

In [54]:
# drop all rows with Status = "None"
lis = lis.dropna(subset=['Status']).reset_index(drop=True)

In [61]:
totals = lis.groupby('Phylum', as_index=False, sort=False).sum()
totals

# empty text-containing columns
totals = totals.drop("Status", axis=1)
totals.insert(0, 'Status', "") 
totals["Genus"] = ""
totals["Species"] = ""

# rename to TOTAL "   "
totals["Phylum"] = totals["Phylum"].str.upper().apply(lambda x: "TOTAL " + x + "S")

# add in line skips
totals = totals.set_index(lis.groupby(['Phylum']).tail(1).index + 0.1)
empty_df = pd.DataFrame("", index=lis.groupby(['Phylum']).tail(1).index+0.2, columns=totals.columns)
totals = pd.concat([totals, empty_df]).sort_index()
totals

1,Status,Phylum,Genus,Species,1/3/19,1/3/19,1/3/19,1/7/19,1/7/19,1/7/19,...,12/6/19,12/6/19,12/6/19,12/5/19,12/16/19,12/16/19,12/16/19,12/4/19,12/4/19,12/4/19
0,,TOTAL DINOFLAGELLATES,,,17952.0,17600.0,30800.0,8800.0,0,17600.0,...,13288.0,13200.0,2904.0,352.0,0,88.0,0,2904.0,704.0,0
1,,TOTAL RAPHIDOPHYTES,,,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,,TOTAL OCHROPHYTE (DIATOMS EXCLUDED)S,,,0,0,0,8800.0,13200.0,8800.0,...,17600.0,17600.0,17600.0,8800.0,22000.0,22000.0,110000.0,22000.0,0,0
3,,TOTAL HAPTOPHYTES,,,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
lis = pd.concat([lis, totals]).sort_index().reset_index(drop=True)

In [20]:
# replace Nans with zero
lis = lis.fillna(0)

In [64]:
# add back multiheader
needed_cols = pd.Series(np.full(len(lis.columns) - len(original_headers), None)) 
original_headers = pd.concat([needed_cols, original_headers.to_series()], ignore_index=True)
lis.columns = pd.MultiIndex.from_arrays([original_headers, lis.columns])

In [22]:
# save dataframe to excel
lis.to_excel(f"outputs/{csv_name}-{str(datetime.now())}.xlsx")